In [18]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("telecom.db")
cur = conn.cursor()

In [2]:
table_query = cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = table_query.fetchall()

In [3]:
tables

[('train',),
 ('event_type',),
 ('severity_type',),
 ('resource_type',),
 ('log_feature',)]

In [36]:
joined = cur.execute("""
            
            SELECT t.*, et.event_type, lf.log_feature, lf.volume, st.severity_type 
            FROM train t
                INNER JOIN event_type AS et ON t.id = et.id
                INNER JOIN log_feature AS lf ON t.id = lf.id
                INNER JOIN severity_type AS st ON t.id = st.id;

""")
joined_data = joined.fetchall()
joineddb = pd.DataFrame(joined_data)

In [37]:
joineddb

,0,1,2,3,4,5,6,7
0,0,14121,location 118,1,event_type 34,feature 232,19,severity_type 2
1,0,14121,location 118,1,event_type 34,feature 312,19,severity_type 2
2,0,14121,location 118,1,event_type 35,feature 232,19,severity_type 2
3,0,14121,location 118,1,event_type 35,feature 312,19,severity_type 2
4,1,9320,location 91,0,event_type 34,feature 235,116,severity_type 2
...,...,...,...,...,...,...,...,...
46726,7379,15189,location 7,0,event_type 11,feature 55,10,severity_type 1
46727,7379,15189,location 7,0,event_type 11,feature 70,1,severity_type 1
46728,7380,17067,location 885,0,event_type 11,feature 191,1,severity_type 1
46729,7380,17067,location 885,0,event_type 11,feature 70,1,severity_type 1


In [38]:
cur.execute("""
            CREATE TABLE joined ('index', 'id', 'location','fault_severity', 'event_type', 'log_feature', 'volume', 'severity_type')
""")

In [39]:
cur.executemany("""
INSERT INTO joined
VALUES (?,?,?,?,?,?,?,?)

""",joineddb.values)

In [42]:
unique_occ = cur.execute("""
            SELECT DISTINCT(event_type), severity_type
            FROM joined

""")
uniquedata = unique_occ.fetchall()
uniquedf = pd.DataFrame(uniquedata)

In [43]:
uniquedf

,0,1
0,event_type 34,severity_type 2
1,event_type 35,severity_type 2
2,event_type 11,severity_type 1
3,event_type 15,severity_type 1
4,event_type 20,severity_type 1
...,...,...
88,event_type 40,severity_type 2
89,event_type 27,severity_type 5
90,event_type 48,severity_type 1
91,event_type 25,severity_type 3


In [51]:
fault_occ = cur.execute("""
            SELECT DISTINCT(fault_severity), COUNT(*) AS count
            FROM joined
            GROUP BY fault_severity;

""")
faultdata = fault_occ.fetchall()
faultdf = pd.DataFrame(faultdata)

In [52]:
faultdf

,0,1
0,0,30249
1,1,11055
2,2,5427
